# Tweet Preprocessing Pipeline – Full Version
Bu notebook, Sentiment140 veri kümesi üzerinde 11 aşamalı NLP ön işleme sürecini adım adım içermektedir.

## Adım 0 – Veri Kümesini Yükleme ve İlk 100 Tweet

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', header=None)
df.columns = ['Sentiment', 'id', 'date', 'query', 'user', 'review']
df = df[['review']].head(100).copy()
df.head()

## Adım 1 – Küçük Harfe Çevirme (Lowercasing)

In [ ]:
df['lowercase'] = df['review'].str.lower()
df[['review', 'lowercase']].head(10)

## Adım 2 – HTML Temizleme

In [ ]:
import re
def remove_html(text):
    return re.sub(r'<.*?>', '', text)
df['no_html'] = df['lowercase'].apply(remove_html)
df[['lowercase', 'no_html']].head(10)

## Adım 3 – URL Temizleme

In [ ]:
def remove_urls(text):
    return re.sub(r'http\S+|www\S+', '', text)
df['no_urls'] = df['no_html'].apply(remove_urls)
df[['no_html', 'no_urls']].head(10)

## Adım 4 – Noktalama Temizleme

In [ ]:
def remove_punctuation(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)
df['no_punct'] = df['no_urls'].apply(remove_punctuation)
df[['no_urls', 'no_punct']].head(10)

## Adım 5 – ChatWords Dönüştürme

In [ ]:
chat_words_map = {
    "u": "you", "r": "are", "idk": "i don't know", "lol": "laughing",
    "btw": "by the way", "omg": "oh my god", "brb": "be right back"
}
def convert_chat_words(text):
    words = text.split()
    return ' '.join([chat_words_map.get(word, word) for word in words])
df['chat_normalized'] = df['no_punct'].apply(convert_chat_words)
df[['no_punct', 'chat_normalized']].head(10)

## Adım 6 – Yazım Düzeltme (İlk 10 Tweet)

In [ ]:
!pip install textblob --quiet
from textblob import TextBlob
def correct_spelling(text):
    return str(TextBlob(text).correct())
df.loc[:9, 'corrected'] = df.loc[:9, 'chat_normalized'].apply(correct_spelling)
df[['chat_normalized', 'corrected']].head(10)

## Adım 7 – Stopword Temizleme

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    if not isinstance(text, str):
        return ''
    words = text.split()
    return ' '.join([word for word in words if word not in stop_words])
df['no_stopwords'] = df['corrected'].apply(remove_stopwords)
df[['corrected', 'no_stopwords']].head(10)

## Adım 8 – Emoji Temizleme

In [ ]:
!pip install emoji --quiet
import emoji
def remove_emojis(text):
    if not isinstance(text, str):
        return ''
    return emoji.replace_emoji(text, replace='')
df['no_emoji'] = df['no_stopwords'].apply(remove_emojis)
df[['no_stopwords', 'no_emoji']].head(10)

## Adım 9 – Tokenization

In [ ]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def tokenize(text):
    try:
        return word_tokenize(str(text))
    except:
        return []
df['tokens'] = df['no_emoji'].apply(tokenize)
df[['no_emoji', 'tokens']].head(10)

## Adım 10 – Stemming

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stem_tokens(tokens):
    return [stemmer.stem(word) for word in tokens]
df['stemmed'] = df['tokens'].apply(stem_tokens)
df[['tokens', 'stemmed']].head(10)

## Adım 11 – Lemmatization

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]
df['lemmatized'] = df['tokens'].apply(lemmatize_tokens)
df[['tokens', 'lemmatized']].head(10)